

# Model testing
In order to test the model we need to find the Region of Interest (ROI), perform the segmentation and then compare the inference with the ground truth in the ROI.

In [50]:
def find_ROI(mask):
    
    min_i = mask.shape[0]
    max_i = 0

    min_j = mask.shape[1]
    max_j = 0

    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i,j]:
                if min_i > i:
                    min_i = i
                if min_j > j:
                    min_j = j
                if max_i < i:
                    max_i = i
                if max_j < j:
                    max_j = j
                    
    return(min_i-15, max_i+15, min_j-15, max_j+15)

In [51]:
def dice_coef(y_true, y_pred):
    volume_sum = y_true.sum() + y_pred.sum()
    if volume_sum == 0:
        return 1.
    volume_intersect = (y_true & y_pred).sum()
    return 2*volume_intersect / volume_sum

In [52]:
def display(display_list):

    plt.figure(figsize=(15,10))

    title = ['Input', 'True Mask ROI', 'Predicted Mask ROI']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    
    plt.show()

In [64]:
test_dir = dir + 'Preprocessed/test/'
test_dir_img = test_dir + 'img/img/'
test_dir_mask = test_dir + 'mask/img/'

accuracy = {}

image = None
mask = None
pred = None

debug=False

model_structure_file = 'UNet_model.json'
model_weights_file = 'UNet_weights.h5'

json_file = open(model_structure_file, 'r')
model_structure = json_file.read()
model = keras.models.model_from_json(model_structure)
model.load_weights(model_weights_file)

In [65]:
start = time.time()

for heart in range(54):
    
    ite = time.time()
    
    accuracy[heart] = {}
    
    for z in range(88):
        
        filename_data = f'heart{str(heart).zfill(3)}_{str(z).zfill(2)}_img.png'
        image = np.array(Image.open(test_dir_img + filename_data))
        
        filename_mask = f'heart{str(heart).zfill(3)}_{str(z).zfill(2)}_mask.png'
        mask = np.array(Image.open(test_dir_mask + filename_mask))
        
        min_y, max_y, min_x, max_x = find_ROI(mask)
        

        image_input = image[np.newaxis, :, :, np.newaxis]
        pred = model.predict(image_input)[0,:,:,0] > 0.5
        
        mask_roi = mask[min_y:max_y, min_x:max_x]>0.5
        pred_roi = pred[min_y:max_y, min_x:max_x]
        
        #print(mask_roi.sum(),pred_roi.sum())
        accuracy[heart][z] = dice_coef(mask_roi, pred_roi)
        
        if debug:
            image.resize(image.shape[0], image.shape[1], 1)
            mask_roi.resize(mask_roi.shape[0], mask_roi.shape[1], 1)
            pred_roi.resize(pred_roi.shape[0], pred_roi.shape[1], 1)
    
    end = time.time()
    
    print(f'Heart {heart}: iteration time elapsed {(end - ite):.3f}s.')

print(f'Total: time elapsed {(end - start):0f}s')

Heart 0: iteration time elapsed 14.698s.
Heart 1: iteration time elapsed 14.678s.
Heart 2: iteration time elapsed 14.496s.
Heart 3: iteration time elapsed 14.299s.
Heart 4: iteration time elapsed 14.521s.
Heart 5: iteration time elapsed 14.488s.
Heart 6: iteration time elapsed 14.231s.
Heart 7: iteration time elapsed 14.571s.
Heart 8: iteration time elapsed 14.422s.
Heart 9: iteration time elapsed 14.244s.
Heart 10: iteration time elapsed 16.173s.
Heart 11: iteration time elapsed 15.083s.
Heart 12: iteration time elapsed 14.679s.
Heart 13: iteration time elapsed 14.517s.
Heart 14: iteration time elapsed 14.772s.
Heart 15: iteration time elapsed 14.444s.
Heart 16: iteration time elapsed 14.441s.
Heart 17: iteration time elapsed 14.654s.
Heart 18: iteration time elapsed 14.468s.
Heart 19: iteration time elapsed 14.925s.
Heart 20: iteration time elapsed 14.839s.
Heart 21: iteration time elapsed 14.772s.
Heart 22: iteration time elapsed 15.039s.
Heart 23: iteration time elapsed 14.741s.
He

In [104]:
accuracy_total = {}
accuracy_mean = {}

for heart in range(len(accuracy)):
    values = np.array(list(accuracy[heart].values()))
    accuracy_mean[heart] = values.mean()


In [105]:
mean_values = np.array(list(accuracy_mean.values()))
total_accuracy_mean = mean_values.mean()
print(total_accuracy_mean)


0.847101924853247
